In [1]:
!pip install mtcnn
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from math import sqrt
from numpy import load, asarray, zeros, ones, savez_compressed
from numpy.random import randn, randint
from skimage.transform import resize
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Flatten , Reshape, Conv2D
from tensorflow.keras.layers import UpSampling2D, AveragePooling2D, LeakyReLU, Layer, Add
from keras.constraints import max_norm
from keras.initializers import RandomNormal
import mtcnn
from mtcnn.mtcnn import MTCNN
from keras import backend
from matplotlib import pyplot
import cv2
import os
from os import listdir
from PIL import Image
import opendatasets as od

In [3]:
od.download("https://www.kaggle.com/datasets/jessicali9530/celeba-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: sayandafadar
Your Kaggle Key: ··········


100%|██████████| 1.33G/1.33G [00:10<00:00, 132MB/s]


In [4]:
# Loading the image file
def load_image(filename):
  image = Image.open(filename)
  image = image.convert('RGB')
  pixels = asarray(image)
  return pixels

# Extract the face from a loaded image and resize
def extract_face(model, pixels, required_size=(128, 128)):
  # Detect the face in the image
  faces = model.detect_faces(pixels)
  if len(faces) == 0:
    return None

  # Extract details of the face
  x1, y1, width, height = faces[0]['box']
  x1, y1 = abs(x1), abs(y1)

  x2, y2 = x1 + width, y1 + height
  face_pixels = pixels[y1:y2, x1:x2]
  image = Image.fromarray(face_pixels)
  image = image.resize(required_size)
  face_array = asarray(image)

  return face_array

# Load images and extract faces for all images in a directory
def load_faces(directory, n_faces):
  # Prepare model
  model = MTCNN()
  faces = list() 

  for filename in os.listdir(directory):
    # Computing the retrival and extraction of faces
    pixels = load_image(directory + filename)
    face = extract_face(model, pixels)
    if face is None:
      continue
    faces.append(face)
    print(len(faces), face.shape)
    if len(faces) >= n_faces:
      break

  return asarray(faces)

In [6]:
# Load and extract all faces
directory = '/content/celeba-dataset/img_align_celeba/img_align_celeba/'
print('Loading faces...')
all_faces = load_faces(directory, 1000)
print('Loaded: ', all_faces.shape)

# Save in compressed format
savez_compressed('img_align_celeba_128.npz')

Loading faces...
2/2 [==============================] - 0s 4ms/step


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 25ms/step
502 (128, 128, 3)
1/1 [==============================] - 0s 31ms/step
503 (128, 128, 3)
1/1 [==============================] - 0s 25ms/step
504 (128, 128, 3)
1/1 [==============================] - 0s 48ms/step
505 (128, 128, 3)
1/1 [==============================] - 0s 46ms/step
506 (128, 128, 3)
1/1 [==============================] - 0s 57ms/step
507 (128, 128, 3)
1/1 [==============================] - 0s 27ms/step
508 (128, 128, 3)
1/1 [==============================] - 0s 32ms/step
509 (128, 128, 3)
1/1 [==============================] - 0s 26ms/step
510 (128, 128, 3)
1/1 [==============================] - 0s 24ms/step
511 (128, 128, 3)
1/1 [==============================] - 0s 24ms/step
512 (128, 128, 3)
1/1 [==============================] - 0s 27ms/step
513 (128, 128, 3)
1/1 [==============================] - 0s 23ms/step
514 (128, 128, 3)
1/1 [==============================] - 

In [8]:
# load the prepared dataset
from numpy import load
data = load('img_align_celeba_128.npz')
faces = data['arr_0']
print('Loaded: ', faces.shape)

KeyError: ignored

In [10]:
data.keys()

KeysView(<numpy.lib.npyio.NpzFile object at 0x7f6fc6e2f880>)

In [ ]:
# Pixel-wise feature vector normalzation layer
class PixelNormalization(Layer):
  # Initialize the layer
  def __init__(self, **kwargs):
    super(PixelNormalization, self).__init__(**kwargs)

  # Perform the operation
  def call(self, inputs):
    # Computing pixel values
    values = inputs**2
    mean_values = backend.mean(values, axis=-1, keepdims=True)
    mean_values += 1e-8
    l2 = backend.sqrt(mean_values)
    normalized = inputs / l2
    return normalized

  # Define the output shape of the layer
  def compute_output_shape(self, input_shape):
      return input_shape

In [ ]:
# mini-batch standard deviation layer
class MinibatchStdev(Layer):
    # initialize the layer
    def __init__(self, **kwargs):
        super(MinibatchStdev, self).__init__(**kwargs)
 
    # perform the operation
    def call(self, inputs):
        mean = backend.mean(inputs, axis=0, keepdims=True)
        squ_diffs = backend.square(inputs - mean)
        mean_sq_diff = backend.mean(squ_diffs, axis=0, keepdims=True)
        mean_sq_diff += 1e-8
        stdev = backend.sqrt(mean_sq_diff)
        
        mean_pix = backend.mean(stdev, keepdims=True)
        shape = backend.shape(inputs)
        output = backend.tile(mean_pix, (shape[0], shape[1], shape[2], 1))
        
        combined = backend.concatenate([inputs, output], axis=-1)
        return combined
 
    # define the output shape of the layer
    def compute_output_shape(self, input_shape):
        input_shape = list(input_shape)
        input_shape[-1] += 1
        return tuple(input_shape)